## Mirrored STDP implements autoencoder learning with spiking neurons
#### Kendra Bubank, Univ. of Chicago, 2015/12

### Learning rule

The autoencoding algorithm is implemeneted in a two-layer network. The input (visible) layer and the hidden layer, which projects back to the input layer. The learning rule is based on the difference between the activity of visible layer due to inputs, and the reconstructed activity from the hidden layer activity as well as the reciprocal feedback connection.

$ \Delta w_{ij} = \eta(x_i - x'_i)y_j $

,where $y_j = \sum_i w_{ij}x_i$, and $x'_{i}= \sum_{j}g(y_jw_{ij})$

The objective function is obtained from the squared error of the predicted output
$\epsilon = ||x_i - x'_i||^2 = \sum_i (x_i - x'_i)^2$

To use the stochastic gradient descent, we need to obtain the dependence of the error $\epsilon$ on weight $w_{ij}$

$\frac{\partial \epsilon}{\partial w_{ij}} = 2\sum_{i}(x_i-x'_i)(\frac{\partial x_i}{\partial w_{ij}}-\frac{\partial x'_i}{\partial w_{ij}})$ -- eq(1)

Given that the input activity $x_i$ is not dependent on the weight $w_{ij}$, $\frac{\partial x_i}{\partial w_{ij}}= 0$

If we denote $g_j'=\frac {\partial g_j}{\partial w_{ij}}$,from the definition, $\frac{\partial x'_i}{\partial w_{ij}} = \sum_j [g_j'\frac {\partial y_j}{\partial w_{ij}}w_{ij}+g_j'\frac {\partial w_{ij}}{\partial w_{ij}}y_j]
= g_j'\frac {\partial y_j}{\partial w_{ij}}w_{ij} + g_j'y_j\delta (w_{ij}) $

Putting back to eq(1), we will have

$\frac{\partial \epsilon}{\partial w_{ij}} = 2\sum_{i}(x_i-x'_i)[0-g_j'\frac {\partial y_j}{\partial w_{ij}}w_{ij} -g_j'y_j\delta (w_{ij})]$


From the equation we can see that the first term depends only on the error message of neuron $i$.Differently, the second term dependents on the errors of all the visible neurons.

### Gradient descent $
-\frac{1}{2}\frac{\partial \epsilon}{\partial w_{ij}}= (x_i-x'_i)g_j'y_j+\sum_{i}(x_{i}-x'_{i})g_j'(\frac {\partial y_j}{\partial w_{ij}}w_{ij}) $ -- eq(2)

Plasticity in the system includes two parts: the inter-layer synaptic strength evolve according to the mirrored STDP. The weights onto the hidden layer then applied synaptic scaling to maintain target neuronal activity.The neuron model is implemented through integrate and fire model, with only the linear inputs - firing rate relationship being important for the network training (Therefore, other more complicated neuron model should give similar learning results)


### Network setting up

Inputs to the visible layer are image patches that have been mean-subtracted. To enbale that the neuron could encode both positive and negative value with only positive firing rates, the author incorporate the ON- and OFF- cells in the visible layers. ON cell will fire at the rate of $max(0,v_{ext})$ and the OFF cells will fire at the rate of $max(0,-V_{ext})$


### Network training

The firing activity of the network could be seperated input three bouts. First, neuron of the visible layer fire at a spike count $x$ given the repeated inputs.  Neurons in the hidden layer fire with spike counts $y$ after a certain delay, which is due to both the synaptic tranduction and the accumulation needed for reaching spike threshold. Eventually, neuron in the visible layer fire again due to the feedback inputs from teh hidden layer with the spike counts $x'$. To prevent the reverbarating growing of the firing activity, a constant scaling factor $\alpha <1$ is applied so that $x'\propto \alpha x $

If we use $W$ to present the weight matrix of visible layer to hidden layer, then the tied-weights along the weak feedback will be maintained when we enforce during the learning the rule $Q = \alpha W^T$

Given that $x'$ is $\alpha$ times smaller, we need to scale $x$ to calculate efficiently the error, which give rise to the scaled reconstruction error  $L_{spike} = ||x-\frac{1}{\alpha}x'||^2$ 

And we will have the updated autoencoder learning rule
### $\Delta w_{ij} = (x_i - \frac{1}{\alpha}x_i')y_i$ -- scaled autoencoder rule

As neurons in the network utilize the leaky - integrate and firing models,their firing rate could be approximated by rectified linear function: $ y = max(0,Wx), x' \approx \alpha W^Ty $
Putting the definition into eq(2), the first term of the gradient descent rule will become $(x'_i>0)(x_i-\frac{1}{\alpha}x'_i)y_j$
, where $(x_i>0)$ is the first order derivatives of function $max(0,Wx)$


The purpose of this paper is then to show that the biological realistic synaptic plasticity rules used by the neurons could implement this scaled autoencoder learning rule both for feedforward and feedaback connections.